In [1]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM,Dense,Input,ConvLSTM2D,Reshape,Activation,Lambda,Softmax,Concatenate
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout,RepeatVector,Reshape,Activation,Lambda,Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import re
import pandas as pd
from keras import backend as K
import keras.backend as kerback
#from keras.models import Model
from tensorflow.keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow as tf
from keras.optimizers import Adam


In [2]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy import random
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import UpSampling2D, Reshape, Activation, Conv2D, BatchNormalization, LeakyReLU, Input, Flatten, multiply,Conv2DTranspose
from keras.layers import Dense, Embedding
from keras.models import Sequential, Model

In [3]:
# supress warnings
import warnings 
warnings.filterwarnings('ignore')

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11711154357016112265
]


In [5]:
img_width, img_height =28,28
img_channel = 1
img_shape = (img_width, img_height, img_channel)
num_classes = 10
z_dim = 100


In [6]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [7]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
    return max_length

In [8]:
list_tweets=prepare_data('C:/tweet.txt')
sequences, tokenizer=tokenizer_data(list_tweets)
vocab_size=len(tokenizer.word_index)
max_length=max_tweet(sequences)

In [9]:
import matplotlib.pyplot as plt
def save_image(epoch):
    r, c = 2,5
    z = np.random.normal(0,1,(r*c, z_dim))
    labels = np.arange(0,10).reshape(-1,1)
    gen_image = generator.predict([z,labels])
    gen_image = 0.5 * gen_image + 0.5
    
    fig, axes = plt.subplots(r,c, figsize = (10,10))
    count = 0
    for i in range(r):
        for j in range(c):
            axes[i,j].imshow(gen_image[count,:,:,0],cmap = 'gray')
            axes[i,j].axis('off')
            axes[i,j].set_title("Digit: %d" % labels[count])
            count+=1
    plt.savefig('images/cgan_%d.jpg' % epoch)
    plt.close()

In [10]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [11]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
        #print("max",max_length)
        #print("seq=",sequences)
        #print(vocab_size)
    return max_length


In [12]:
def build_generator(latent_dim=100):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(vocab_size, 100)(in_label)
    # linear multiplication
    n_nodes = 57 * 1
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((57, 1, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 66 * 57 * 1
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((57, 1, 66))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(66, (1,1), strides=(1,1), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(66, (1,1), strides=(1,1), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (57,1), activation='tanh', padding='same')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model
generator = build_generator()
generator.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3762)         379962      input_2[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 100)       84100       input_1[0][0]                    
_______________________________________________________________________________________

In [13]:
from keras.layers import Dropout, Concatenate
import numpy as np

def define_discriminator(in_shape=(57,1,1)):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(vocab_size, 50)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(66, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(66, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model
discriminator = define_discriminator()
discriminator.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 50)        42050       input_3[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1, 57)        2907        embedding_1[0][0]                
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 57, 1, 1)]   0                                            
_______________________________________________________________________________________

In [14]:
from keras.optimizers import Adam
discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(0.0002, 0.5), metrics = ['accuracy'])

z = Input(shape=(z_dim,))
label = Input(shape= (1,))
img = generator([z,label])

discriminator.trainable = False
prediction = discriminator([img, label])

cgan = Model([z, label], prediction)
cgan.compile(loss= 'binary_crossentropy', optimizer = Adam(0.0002, 0.5))

In [40]:
def sample_tweet(noise,sampled_labels,g_model):
    result = g_model.predict([noise,sampled_labels])
    #print("1",result)
    #print("2",result[0])
    #print(len(result))#66
    #print(result)
    #print(len(result))
    #result=result[0]
    #print(len(result))#57
    #print(result)
    #print(result[0][0][0][0])
    #print(result[0][1][0][0])
    #print(result[1][0][0][0])
    res=[]
    
    #print(result[0])
    #for i in range(len(result)):
    #print(len(result[0]))
    predicted_word =""
    for j in range(len(result[0])):
        #predicted_word = predicted_word+' '+tokenizer.decode(result[0][j][0][0],skip_special_tokens=True)
        #predicted_word = ''
        for word, index in tokenizer.word_index.items():
            #print(result[i][j][0][0]*100)
            if index == abs(int(result[0][j][0][0]*10)):
                #print(result[i][j][0][0]*10)
                #print(word)
                predicted_word = predicted_word+' '+word
                #print(predicted_word)  
                break
    l=predicted_word.split()
    #print(len(l))    
    res.append(predicted_word)
    print(res)
    
    #for i in res:
        #print(i)
    return res

In [41]:
def disc_text(model_disc, seed_text):
    #print(seed_text)
    sequences, tokenizer=tokenizer_data(seed_text)
    sequences= pad_sequences(sequences, maxlen=max_length, padding='post')
    y = model_disc.predict(sequences)
    #print(y)
    #for i in y:
    if(i<0.7):
        print("fake")
    else:
        print("real")

In [42]:
def generate_real_samples():
    list_tweets=prepare_data('C:/tweet.txt')
    sequences, tokenizer=tokenizer_data(list_tweets)
    max_length=max_tweet(sequences)
    sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
    y=sequences[:,-1]
    y_train = y.reshape(-1, 1)
    X=sequences
    X_train = np.expand_dims(X, axis=2)
    print("x",X_train.shape)
    nb_tweet=len(sequences)
    batch_size=max_length
    
    return [X_train,y_train]


In [43]:
def train(epochs, batch_size, save_interval):
    #(X_train, y_train), (_, _) = mnist.load_data()
    #list_tweets=prepare_data('C:/tweet.txt')
    #sequences, tokenizer=tokenizer_data(list_tweets)
    #vocab_size=len(tokenizer.word_index)
    #max_length=max_tweet(sequences)
    #X_train =sequences[:]
    #print(X_train.shape)
    #X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
    #print("X=",X_train)
    #print(X_train.shape)
    [X_train,y_train]= generate_real_samples()
    #X_train = np.expand_dims(X_train, axis=1)
    #print("X=",X_train)
    #print(X_train.shape)
    #print("X=",X_train[0])
    #print(X_train[0].shape)
    #y =sequences[33:]
    #y = pad_sequences(y, maxlen=max_length, padding='post')
    #print("y=",y)
    #print(y.shape)
    #y = np.expand_dims(y, axis=1)
    #print(X_train)
    #print(y)
    #print('train_x shape:', X_train.shape)
    #print('train_y shape:', y.shape)
    #X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.33)
    #print(X_test.shape)
    #print(Y_test.shape)
    #print(X_train.shape)
    #print(Y_train.shape)
    #print("x_train",X_train)
    #print("x_train[0]",X_train[0])
    #print(X_train[0].shape)#(28,28)
    #print(X_train.shape)#(60000,28,28) 60000 images size mte3hom (28,28) donc twali 22 tweet size mte3hom (1,57)
    #y_train = np.random.randint(0, X_train.shape[0], 66)
    #sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
    #y_train=sequences[:,-1]
    #print("y_train",y_train)
    #print(y_train.shape)#(60000) tweeta bark
    X_train = (X_train - 127.5) / 127.5
    #print(X_train.shape)
    X_train = np.expand_dims(X_train, axis=3)
    #print(X_train.shape)#(60000,28,28,1)
    real = np.ones(shape=(batch_size, 1))
    #print("real_shape",real.shape)#(128,1)
    fake = np.zeros(shape=(batch_size, 1))
    #print("fake_shape",fake.shape)#(128,1)
    
    for iteration in range(epochs):
        
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        #print("idx",idx)
        #print(idx.shape)#(128,)
        imgs, labels = X_train[idx], y_train[idx]
        #print("img",imgs)
        #print(imgs.shape)#(none,28,28,1)
        #print("labels",labels)
        #print(labels.shape)#(128,)
        
        z = np.random.normal(0, 1, size=(batch_size, z_dim))
        #print("z",z)
        #print(z.shape)#(128,100)
        gen_imgs = generator.predict([z, labels],verbose=0)
        
        #print("gen_imgs",gen_imgs)
        #print(gen_imgs.shape)#(128,28,28,1)
        
        d_loss_real = discriminator.train_on_batch([imgs, labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        z = np.random.normal(0, 1, size=(batch_size, z_dim))
        #print("z=",z)
        #print(z.shape)#(128,1)
        labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
        #print("labels",labels)
        #print(labels.shape)#(128,1)
        
        
        g_loss = cgan.train_on_batch([z, labels], real)
        
        #if iteration % save_interval == 0:
        tweet=sample_tweet(z, labels, generator)
        
        #print(tweet)
        #disc_text(discriminator,[tweet, labels] )
        print('{} [D loss: {}, accuracy: {:.2f}] [G loss: {}]'.format(iteration, d_loss[0], 100 * d_loss[1], g_loss))
        #save_image(iteration)

In [44]:

# train network
train(300, 66, 1000)

x (66, 57, 1)
[' this it it it it this it this this it this it it it it this this it it our it it this this it it this the it this it this the this this this this it it this it it it it it it it it it it it it it it it it it']
0 [D loss: 0.09117848332971334, accuracy: 98.48] [G loss: 0.34978923201560974]
[' this it this it it you it it this it it it it this it it this it it this it it it this this it it this it it you this this it this it this this it it it it it it this it it it it it it it it it it it it']
1 [D loss: 0.09128336608409882, accuracy: 97.73] [G loss: 0.38815876841545105]
[' this this this it this it it it it it it it it it it it this it it it it this it it it it it this this this this this you this this it this this it this it this it it it it it it it it it it it it it this this']
2 [D loss: 0.0695074126124382, accuracy: 97.73] [G loss: 0.32041022181510925]
[' this it this it it this it it it it this it it it this it it it it this it this you it it this it this it this 